# **Regression Notebook**

## Objectives

* Fetch Data from Kaggle and save as raw file and unzip.
* Inspect the data and save it under inputs/datasets/raw
* Inspect the data and save it under outputs/datasets/collection

## Inputs

* Kaggle JSON file - the authentication token.

## Outputs

* Generate Dataset: outputs/datasets/collection/trimmed_covid_dataset.csv

## Additional Comments

* In case you have any additional comments that don't fit in the previous bullets, please state them here. 

---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

---

## Load Data

---

## MP Pipeline: Regressor

### Create ML pipeline

Custom Class for hyperparameter optimisation

### Split Train Test Set

### Grid Search CV - Sklearn

#### Use default hyperparameters to find most suitable algorithm

hyperparameter optimisation search using default hyperparameters

Check Results

#### Do an extensive search on the most suitable model to find the best hyperparameter configuration.

Define model and parameters, for Extensive Search

Extensive GridSearch CV

Check Results

Check the best model

Parameters for best model

Define the best regressor, based on search

### Assess feature importance

### Evaluate on Train and Test Sets

Evaluate Performance

### Regressor with PCA

Let's explore potential values for PCA n_components.

Apply PCA separately to the scaled data

### Rewrite ML Pipeline for Modelling

### Grid Search CV – Sklearn

#### Use standard hyperparameters to find the most suitable model.

Do a quick optimisation search

Check results

#### Do an extensive search on the most suitable model to find the best hyperparameter configuration.

Define model and parameters for extensive search

Extensive GridSearch CV

Check results

Check the best model

Parameters for best model

Define the best regressor

### Evaluate Regressor on Train and Tests Sets

## Convert Regression to Classification

### Convert numerical target to bins, and check if it is balanced

### Rewrite ML Pipeline for Modelling

### Load algorithms for classification

### Split Train Test Sets

### Grid Seach CV – Sklearn

#### Use standard hyper parameters to find most suitable model

GridSearch CV

Check results

#### Do an extensive search on the most suitable model to find the best hyperparameter configuration.

Define models and parameters

Extensive GridSearch CV

Check results

Check the best model

Parameters for best model

- We are saving this content for later

Define the best clf pipeline

### Assess feature importance

We can assess feature importance for this model with .feature_importances_

### Evaluate Classifier on Train and Test Sets

Custom Function

List that relates the classes and tenure interval

We can create manually

### Which pipeline to choose?

We fitted 3 pipelines:

- Regression
- Regression with PCA
- Classifier

The regressor pipelines didn't reach the expected performance threshold (0.7 R2 score) for the train and test set.

The classifier was tuned on Recall for class 0 (tenure <4 months), since we are interested to detect prospects that may churn soon.

- It has reasonable performance for class 0 (<4 months) and class 2 (+20 months)
- Class 1 (4 to 20 months) has weak performance.

### Refit pipeline with best features

#### Rewrite Pipeline

### Split Train Test Set, only with best features

Subset Best Features

#### Grid Search CV – Sklearn

We are using the same model from the previous GridCV search

And the best parameters from the previous GridCV search

You will need to type in manually since the hyperparameter values have to be a list. The previous dictionary is not in this format.

GridSearch CV

Check results

Check the best model

Define the best clf pipeline

### Assess feature importance

### Evaluate Classifier on Train and Test Sets

### Push files to the repo

We will generate the following files

- Train set
- Test set
- Modeling pipeline
- label map
- features importance plot

### Train Set: features and target

### Test Set: features and target

### Modelling pipeline

ML pipeline for predicting tenure

### List mapping target levels to ranges

Map for converting numerical variable to categorical variable

### Feature importance plot